# Data Augmentation Model

- Train: 75Agree_train
- Test: 75Agree_test

# TODO

- missing cv for hyp tunning

- organize better the code

- data augmentation hyperparameters (tune?) and get better examples

In [1]:
import nltk
import spacy
import torch
import random
import numpy as np
import pandas as pd
import seaborn as sns
from nltk.corpus import wordnet
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from sklearn.model_selection import StratifiedKFold
from transformers import MarianMTModel, MarianTokenizer
from sklearn.metrics import classification_report, confusion_matrix
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

#spacy.cli.download("en_core_web_sm")
nltk.download("wordnet")

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "figure.figsize": (5, 5*0.8),
    "savefig.dpi": 300,
})

/opt/anaconda3/envs/CAA02/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package wordnet to /Users/hugover/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Step 1: Read the original data

In [2]:
# load the training and test datasets
df_train = pd.read_csv('data/75Agree_train.csv')
df_test = pd.read_csv('data/75Agree_test.csv')

# label encoders
label2id = {"negative": 0, "neutral": 1, "positive": 2}
id2label = {v: k for k, v in label2id.items()}

# label encoding
df_train["label"] = df_train["label"].map(label2id)
df_test["label"] = df_test["label"].map(label2id)

# balance the training set: undersample to minority class
df_train = df_train.groupby("label").apply(lambda x: x.sample(df_train["label"].value_counts().min())).reset_index(drop=True)
df_train.value_counts("label")

/var/folders/sz/96b_h5gn3y33k0c7jsvrstmm0000gn/T/ipykernel_61448/918004758.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_train = df_train.groupby("label").apply(lambda x: x.sample(df_train["label"].value_counts().min())).reset_index(drop=True)


label
0    336
1    336
2    336
Name: count, dtype: int64

### Step 2: Create the data augmentation pipeline

In [3]:
class AugmentedTextDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=128,
                 p_back=0.3, p_syn=0.4, p_ner=0.2, p_none=0.3):
        self.df = df.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.p_back = p_back
        self.p_syn = p_syn
        self.p_ner = p_ner
        self.p_none = p_none

        self.nlp = spacy.load("en_core_web_sm")

        en_de_model_name = "Helsinki-NLP/opus-mt-en-de"
        de_en_model_name = "Helsinki-NLP/opus-mt-de-en"
        self.en_de_tok = MarianTokenizer.from_pretrained(en_de_model_name)
        self.de_en_tok = MarianTokenizer.from_pretrained(de_en_model_name)
        self.en_de_mod = MarianMTModel.from_pretrained(en_de_model_name)
        self.de_en_mod = MarianMTModel.from_pretrained(de_en_model_name)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.loc[idx, 'sentence']
        label = self.df.loc[idx, 'label']

        # Randomly apply augmentation techniques based on probabilities
        if not random.random() < self.p_none:
            if random.random() < self.p_back:
                text = self.back_translate(text) or text
            if random.random() < self.p_syn:
                text = self.synonym_replace(text) or text
            if random.random() < self.p_ner:
                text = self.ner_replace(text) or text

        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors="pt"
        )
        item = {k: v.squeeze() for k, v in encoding.items()}
        item["labels"] = torch.tensor(label)
        return item

    def back_translate(self, text):
        try:
            de = self.translate(text, self.en_de_tok, self.en_de_mod)
            return self.translate(de, self.de_en_tok, self.de_en_mod)
        except:
            return None

    def translate(self, text, tokenizer, model):
        batch = tokenizer([text], return_tensors="pt", padding=True, truncation=True)
        gen = model.generate(**batch)
        return tokenizer.decode(gen[0], skip_special_tokens=True)

    def get_synonyms(self, word):
        syns = set()
        for syn in wordnet.synsets(word):
            for l in syn.lemmas():
                w = l.name().replace('_', ' ')
                if w.lower() != word.lower():
                    syns.add(w)
        return list(syns)

    def synonym_replace(self, text, p=0.2):
        words = text.split()
        out = []
        for word in words:
            if random.random() < p:
                syns = self.get_synonyms(word)
                if syns:
                    out.append(random.choice(syns))
                    continue
            out.append(word)
        return ' '.join(out)

    def ner_replace(self, text):
        doc = self.nlp(text)
        template, pool = self.extract_template(doc)
        if not pool:
            return None
        return self.fill_template(template, pool)

    def extract_template(self, doc):
        entity_pool = {}
        template = doc.text
        offsets = []
        for ent in doc.ents:
            if ent.label_ in ["ORG", "DATE", "EVENT"]:
                tag = f"<{ent.label_}>"
                offsets.append((ent.start_char, ent.end_char, tag, ent.text, ent.label_))
                if ent.label_ not in entity_pool:
                    entity_pool[ent.label_] = set()
                entity_pool[ent.label_].add(ent.text)

        offsets.sort()
        new_text = ""
        last = 0
        for start, end, tag, _, _ in offsets:
            new_text += template[last:start] + tag
            last = end
        new_text += template[last:]
        return new_text, entity_pool

    def fill_template(self, template, entity_pool):
        out = template
        for label, values in entity_pool.items():
            tag = f"<{label}>"
            if tag in out and values:
                out = out.replace(tag, random.choice(list(values)))
        return out

In [4]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
def tokenize_function(examples):
    return tokenizer(examples["sentence"], truncation=True, padding="max_length", max_length=128)

# apply the augmentation
dataset = AugmentedTextDataset(df_train, tokenizer, p_none=0)

# get some examples of the augmentation
for i in range(5):
    original = df_train.loc[i, 'sentence']
    augmented = dataset[i]
    decoded = tokenizer.decode(augmented['input_ids'], skip_special_tokens=True)
    print(f"{original}\n{decoded}\n")

The operating margin of Aker Yards Cruise & Ferries division went down from 8.3 % to 6.4 % in the first quarter of 2007 .
the operating security deposit of aker yards cruise & ferries division went down from 8. 3 % to 6. 4 % in the first tail of 2007.

The total capital of funds managed by the bank decreased by 28 % to EUR 284mn by the end of September 2008 .
the total capital of funds managed by the bank decreased by 28 % to eur 284mn by the end of september 2008.

In Finland , snow storms brought trees down on power lines , cutting off electricity for some 2,000 households .
in finland, snowstorms brought down trees on power lines, which thinned off some 2, 000 households'electricity.

EMSA Deputy Chairman of the Board Juri Lember told BNS on Wednesday that this was the first time he heard about the strike as the Swedish side had not informed the Estonian union yet .
emsa deputy chairman of the board juri lember told bns on wednesday that this was the first time he heard about the st

### Step 3: Cross validation for hyperparameter tuning

In [5]:
# tokenization
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
def tokenize_function(examples):
    return tokenizer(examples["sentence"], truncation=True, padding="max_length", max_length=128)

# prepare for cv
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# set random search area
def get_hyperparameters():
    return {
        "num_train_epochs": int(np.random.choice([1, 2, 3, 4, 5])), #
        "learning_rate": float(np.random.choice([1e-5, 1e-4, 1e-3, 1e-2])), #
        "weight_decay": float(np.random.choice([0.0, 0.01, 0.05, 0.1, 0.5])), #
    }

# get the cv results dataframe or create a new one
try:
    all_results = pd.read_csv("model BERT (2).csv")
except FileNotFoundError:
    all_results = pd.DataFrame(columns=["num_train_epochs", "learning_rate", "weight_decay", "eval_loss"])

# start the hyperparameter tuning
for _ in range(0): #range(25)
    current_hyperparameters = get_hyperparameters()
    current_hyperparameters_eval_loss = 0
    current_results = {"num_train_epochs": [], "learning_rate": [], "weight_decay": [], "eval_loss": []}

    # make sure the hyperparameters are not already evaluated
    mask = (
        (all_results['num_train_epochs'] == current_hyperparameters['num_train_epochs']) &
        (all_results['learning_rate'] == current_hyperparameters['learning_rate']) &
        (all_results['weight_decay'] == current_hyperparameters['weight_decay'])
    )
    if mask.any():
        print("Hyperparameters already evaluated, skipping...")
        continue
    else:
        print(f"Evaluating hyperparameters: {current_hyperparameters}")
    
    # tuning hyperparameters
    for fold, (train_idx, val_idx) in enumerate(skf.split(df_train, df_train["label"])):

        # get the folds
        df_fold_train = df_train.iloc[train_idx]
        df_fold_val = df_train.iloc[val_idx]

        # dataset preparation for huggingface transformers
        train_dataset = AugmentedTextDataset(df_fold_train, tokenizer)
        val_dataset = AugmentedTextDataset(df_fold_val, tokenizer, p_none=1)

        # the model
        model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3, id2label=id2label, label2id=label2id)

        # hyperparameters
        training_args = TrainingArguments(
            num_train_epochs=current_hyperparameters["num_train_epochs"],
            learning_rate=current_hyperparameters["learning_rate"],
            weight_decay=current_hyperparameters["weight_decay"],
            eval_strategy="no",
            logging_strategy="no",
            report_to=None,
            dataloader_pin_memory=False,
            per_device_train_batch_size=16,
            per_device_eval_batch_size=16,
            save_total_limit=1,
            save_steps=500,
            output_dir=f"./results/fold_{fold}",
            disable_tqdm=True,
        )

        # training the model
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            tokenizer=tokenizer,
        )
        trainer.train()

        # update the evaluation loss
        current_hyperparameters_eval_loss += trainer.evaluate(eval_dataset=val_dataset)["eval_loss"]

    # update the results dataframe
    current_results["num_train_epochs"].append(current_hyperparameters["num_train_epochs"])
    current_results["learning_rate"].append(current_hyperparameters["learning_rate"])
    current_results["weight_decay"].append(current_hyperparameters["weight_decay"])
    current_results["eval_loss"].append(current_hyperparameters_eval_loss / skf.n_splits)
    all_results = pd.concat([all_results, pd.DataFrame(current_results)], ignore_index=True)
    all_results.to_csv("model BERT (2).csv", index=False)

"Hyperparameter tuning completed. Results saved to model BERT (2).csv."

'Hyperparameter tuning completed. Results saved to model BERT (2).csv.'

In [6]:
all_results = all_results.sort_values(by="eval_loss").reset_index(drop=True)
all_results.to_csv("model BERT (2).csv", index=False)

all_results.head(3)

,num_train_epochs,learning_rate,weight_decay,eval_loss
0,2,0.00010,0.10,0.303190
1,3,0.00010,0.10,0.306246
2,5,0.00001,0.01,0.400804


### Step 4: Create the model with the best hyperparameters

In [7]:
best_hyperparameters = all_results.iloc[0]
print(f"Best hyperparameters found in {all_results.shape[0]} tested combinations:")
print(f"Num train epochs: {best_hyperparameters['num_train_epochs']}")
print(f"Learning rate: {best_hyperparameters['learning_rate']}")
print(f"Weight decay: {best_hyperparameters['weight_decay']}")

Best hyperparameters found in 15 tested combinations:
Num train epochs: 2.0
Learning rate: 0.0001
Weight decay: 0.1


In [ ]:
# tokenization
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
def tokenize_function(examples):
    return tokenizer(examples["sentence"], truncation=True, padding="max_length", max_length=128)

# dataset preparation for huggingface transformers
train_dataset = AugmentedTextDataset(df_train, tokenizer)
test_dataset = AugmentedTextDataset(df_test, tokenizer, p_none=1)

# the model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3, id2label=id2label, label2id=label2id)

training_args = TrainingArguments(
    num_train_epochs=best_hyperparameters['num_train_epochs'],
    learning_rate=best_hyperparameters['learning_rate'],
    weight_decay=best_hyperparameters['weight_decay'],
    eval_strategy="steps",
    eval_steps=25,
    logging_strategy="steps",
    logging_steps=25,
    report_to=None,
    dataloader_pin_memory=False,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    save_total_limit=1,
    save_steps=500,
    output_dir="./results",
)

# train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
)
trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/var/folders/sz/96b_h5gn3y33k0c7jsvrstmm0000gn/T/ipykernel_61448/2065444062.py:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# get training and evaluation loss history
log_history = trainer.state.log_history
train_loss = [x["loss"] for x in log_history if "loss" in x]
eval_loss = [x["eval_loss"] for x in log_history if "eval_loss" in x]
eval_steps = [x["step"] for x in log_history if "eval_loss" in x]
train_steps = [x["step"] for x in log_history if "loss" in x]

plt.figure(figsize=(8, 5))
plt.plot(train_steps, train_loss, label="Training Loss")
plt.plot(eval_steps, eval_loss, label="Validation Loss")
plt.xlabel("Steps")
plt.ylabel("Loss")
plt.title("Learning Curve for Data Augmented BERT Model")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig("assets/data_augmented_bert_learninc_curve.png", transparent=True)
plt.show()

### Step 5: Evaluate the model

In [ ]:
train_dataset = AugmentedTextDataset(df_train, tokenizer, p_none=1)
test_dataset = AugmentedTextDataset(df_test, tokenizer, p_none=1)

def evaluate_dataset(dataset, split_name):
    output = trainer.predict(dataset)
    preds = np.argmax(output.predictions, axis=1)
    y_true = output.label_ids

    print(f"\n=== Classification Report ({split_name}) ===")
    print(classification_report(y_true, preds, target_names=["negative", "neutral", "positive"]))

    cm = confusion_matrix(y_true, preds, normalize='true')
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt=".2f", cmap="Blues",
                xticklabels=["negative", "neutral", "positive"],
                yticklabels=["negative", "neutral", "positive"])
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title(f"Confusion Matrix ({split_name})")
    plt.tight_layout()
    plt.savefig(f"assets/dataaugmented_bert_confusion_matrix_{split_name}.png", transparent=True)
    plt.show()

# evaluate the model on the training and test datasets
evaluate_dataset(train_dataset, "Train")
evaluate_dataset(test_dataset, "Test")